In [16]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import os
!pip install tensorflow-addons==0.20.0 # Make sure this version is compatible with your tensorflow

In [2]:
# **Enable Mixed Precision for Faster Training**
tf.keras.mixed_precision.set_global_policy('mixed_float16')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# **Define Hyperparameters**
BATCH_SIZE = 32
IMG_SIZE = 224
EPOCHS = 10
LR = 3e-5
DATA_DIR = "/content/drive/My Drive/Facial_Emotion_Recognition_Project/FER_25"  # dataset path

In [6]:
# **Load and Preprocess Data**
def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.image.convert_image_dtype(image, tf.float32)  # Normalize [0,1]
    return image, label

In [7]:
# **Load Full Dataset (Since No Validation Folder Exists)**
full_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR,
    label_mode="int",
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=True
)

Found 7210 files belonging to 2 classes.


In [8]:
# **Split Dataset (90% Train, 10% Validation)**
dataset_size = full_ds.cardinality().numpy()  # Total number of batches
val_size = int(0.1 * dataset_size)  # 10% for validation

train_ds = full_ds.skip(val_size)  # Take 90% for training
val_ds = full_ds.take(val_size)    # Take 10% for validation

In [9]:
# **Optimize Dataset Performance**
train_ds = train_ds.map(preprocess).cache().shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(preprocess).cache().batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [17]:
# **Load ViT-Base Model**
import tensorflow_addons as tfa
base_model = tfa.applications.ViT(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    pooling='avg'
)

/usr/local/lib/python3.11/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/usr/local/lib/python3.11/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.13.0 and strictly below 2.16.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.18.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're u

AttributeError: module 'tensorflow.keras.layers' has no attribute 'AbstractRNNCell'